In [84]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [85]:
%sql sqlite:///lab2.db

%sql DROP TABLE Sailors
%sql CREATE TABLE Sailors (sid INTEGER PRIMARY KEY,sname TEXT,rating INTEGER,age INTEGER)

%sql INSERT INTO Sailors (sid, sname, rating, age) VALUES (1, 'John', 8, 25),(2, 'Jane', 7, 28),(3, 'Mark', 6, 30),(4, 'Emily', 8, 22),(5, 'Michael', 7, 27),(6, 'Sarah', 7, 26),(7, 'Alex', 6, 29),(8, 'Emma', 8, 24),(9, 'Michaela', 7, 31),(10, 'David', 9, 23)

%sql DROP TABLE Boats
%sql CREATE TABLE Boats (bid INTEGER PRIMARY KEY,name TEXT,color TEXT)



%sql INSERT INTO Boats (bid, name, color) VALUES (101, 'Boat1', 'Red'),(102, 'Boat2', 'Blue'),(103, 'Boat3', 'Red'),(104, 'Boat4', 'Green'),(105, 'Boat5', 'Blue'),(106, 'Boat6', 'Red'),(107, 'Boat7', 'Blue'),(108, 'Boat8', 'Red'),(109, 'Boat9', 'Blue'),(110, 'Boat10', 'Green')
%sql DROP TABLE Reserves

%sql CREATE TABLE Reserves (sid INTEGER,bid INTEGER,day DATE,PRIMARY KEY (sid, bid, day),FOREIGN KEY (sid) REFERENCES Sailors (sid),FOREIGN KEY (bid) REFERENCES Boats (bid))


%sql INSERT INTO Reserves (sid, bid, day) VALUES (1, 101, '2023-06-01'),(2, 110, '2023-06-02'),(3, 102, '2023-06-03'),(4, 103, '2023-06-04'),(5, 105, '2023-06-05'),(6, 102, '2023-06-06'),(7, 103, '2023-06-07'),(8, 105, '2023-06-08'),(9, 101, '2023-06-09'),(10, 104, '2023-06-10'),(11, 102, '2023-06-11'),(12, 105, '2023-06-12'),(13, 101, '2023-06-13'),(14, 107, '2023-06-14'),(15, 108, '2023-06-15'),(16, 106, '2023-06-16'),(17, 109, '2023-06-17'),(18, 110, '2023-06-18'),(19, 106, '2023-06-19'),(20, 108, '2023-06-20')

 * sqlite:///lab2.db
Done.
 * sqlite:///lab2.db
Done.
 * sqlite:///lab2.db
10 rows affected.
 * sqlite:///lab2.db
Done.
 * sqlite:///lab2.db
Done.
 * sqlite:///lab2.db
10 rows affected.
 * sqlite:///lab2.db
Done.
 * sqlite:///lab2.db
Done.
 * sqlite:///lab2.db
20 rows affected.


[]

In [86]:
%sql SELECT s.sname  FROM Sailors s, Reserves r WHERE r.bid = 110 and r.sid = s.sid

 * sqlite:///lab2.db
Done.


sname
Jane


In [87]:
%sql SELECT s.sname FROM Sailors s, Reserves r, Boats b WHERE b.color = 'Red' and b.bid = r.bid and r.sid = s.sid


 * sqlite:///lab2.db
Done.


sname
John
Emily
Alex
Michaela


In [88]:
%sql SELECT s.sname FROM Sailors s, Reserves r, Boats b WHERE b.color in ('Red','Blue') and b.bid = r.bid and r.sid = s.sid

 * sqlite:///lab2.db
Done.


sname
John
Mark
Emily
Michael
Sarah
Alex
Emma
Michaela


In [89]:
%%sql 
SELECT s.sname
FROM Sailors s
WHERE EXISTS (
  SELECT *
  FROM Reserves r1
  INNER JOIN Boats b1 ON r1.bid = b1.bid
  WHERE r1.sid = s.sid AND b1.color = 'Red'
) AND EXISTS (
  SELECT *
  FROM Reserves r2
  INNER JOIN Boats b2 ON r2.bid = b2.bid
  WHERE r2.sid = s.sid AND b2.color = 'Blue'
)


 * sqlite:///lab2.db
Done.


sname


In [80]:
%%sql
SELECT s.sname
FROM Sailors s
LEFT JOIN Reserves r ON s.sid = r.sid
LEFT JOIN Boats b ON r.bid = b.bid
GROUP BY s.sid
HAVING COUNT(*) = SUM(CASE WHEN b.color = 'Blue' THEN 1 ELSE 0 END);

 * sqlite:///lab2.db
Done.


sname
Mark
Michael
Sarah
Emma


In [90]:
%%sql
SELECT rating, AVG(age) AS average_age
FROM Sailors
GROUP BY rating
HAVING COUNT(*) >= 2;

 * sqlite:///lab2.db
Done.


rating,average_age
6,29.5
7,28.0
8,23.666666666666668


In [82]:
%%sql
SELECT s.sname, AVG(reservations) AS average_reservations
FROM Sailors s
LEFT JOIN (
  SELECT sid, COUNT(*) AS reservations
  FROM Reserves
  GROUP BY sid
) r ON s.sid = r.sid
GROUP BY s.sid;

 * sqlite:///lab2.db
Done.


sname,average_reservations
John,1.0
Jane,1.0
Mark,1.0
Emily,1.0
Michael,1.0
Sarah,1.0
Alex,1.0
Emma,1.0
Michaela,1.0
David,1.0


In [91]:
%%sql
SELECT b.name
FROM Boats b
INNER JOIN (
  SELECT bid, COUNT(*) AS reservations
  FROM Reserves
  GROUP BY bid
  ORDER BY reservations DESC
  LIMIT 1
) r ON b.bid = r.bid;

 * sqlite:///lab2.db
Done.


name
Boat5
